# Demonstration of density estimation.

Here, we demonstrate how to use the density estimation module of `pygwb` with simulated time-frequency maps.

In [ ]:
%pylab inline

from itertools import combinations

import bilby
from gwpy.spectrogram import Spectrogram
from gwpy.timeseries import TimeSeries

from pygwb import spectral

Simulate some data

- simulate 4096s of data in three interferometers following design sensitivities.
- add a coherent signal centered at 200 Hz. Note that this ignores the actual overlap between different interferometers.

In [ ]:
ifos = bilby.gw.detector.InterferometerList(["H1", "L1", "V1"])

chunk_duration = 2048 * 2
sampling_frequency = 1024

ifos.set_strain_data_from_power_spectral_densities(
    duration=chunk_duration, sampling_frequency=sampling_frequency
)

coherent_psd = bilby.gw.detector.PowerSpectralDensity.from_amplitude_spectral_density_array(
    frequency_array=ifos[0].frequency_array,
    asd_array=1e-23 * np.exp(-(ifos[0].frequency_array - 200) ** 2 / 2)
)
common_noise = coherent_psd.get_noise_realisation(
    duration=chunk_duration, sampling_frequency=sampling_frequency
)[0]
common_noise = bilby.core.utils.infft(common_noise, sampling_frequency=sampling_frequency)

### Convert to "fftgrams" with `gwpy`

In [ ]:
segment_duration = 32
psd_duration = 32
frequency_resolution = 1 / 4
overlap = segment_duration / 2

In [ ]:
gwpy_data = dict()

for ifo in ifos:
    gwpy_data[ifo.name] = TimeSeries(
        ifo.time_domain_strain + common_noise, times=ifo.time_array
    ).fftgram(fftlength=segment_duration, overlap=overlap)
    np.log10(abs(gwpy_data[ifo.name])).plot(ylim=(20, 500))
    plt.show()
    plt.close()

### Generate PSD and CSD spectrograms

- combine fftgrams
- coarsen in time and frequency

_FIXME: verify this_

Here we use a running mean average in time to generate a Welch average for each segment covering `psd_duration`s ($T_{P}$) of data starting from the beginning of the first segment.

The first segment ends duration $D$s after the beginning of the first.
The $n$th segments ends $D + (n - 1) T_{o}$s after the beginning of the first.
This means that we require $T_{P} = D + (n - 1) T_{o}$ or $n = \frac{T_{P} - D}{T_{o}} + 1$ segments.

The $n$th segment then ends $n T_{O}$s after the first segment ($T_{o}$ is the overlap time).
This means that we require $T_{P} = D + n T_{o}$ or $n = \frac{T_{P} - D}{T_{o}}$ segments.

After this, we compute the final PSD using a before/after.

For the CSD, we then need the last segment used to estimate the PSD to be entirely non-overlapping with the corresponding CSD segment.

This means that the CSD segment starts at the earliest $T_{P} + D$s after the PSD estimation segment.
I.e., $n' \geq n + \frac{D}{T_{o}} = \frac{T_{P}}{T_{o}}$.

The "after" PSD estimate must start at $n_{a} = T_{p} + D$

In [ ]:
psd_grams = dict()

for ifo in gwpy_data:
    temp = abs(spectral.coarse_grain_spectrogram(
        spectral.density(gwpy_data[ifo], gwpy_data[ifo]),
        delta_t=psd_duration,
        delta_f=frequency_resolution,
        time_method="running_mean"
    ))
    psd_grams[ifo] = spectral.before_after_average(temp, segment_duration, psd_duration)
    np.log10(psd_grams[ifo]).plot(ylim=(20, 500))
    plt.show()
    plt.close()
    
    plt.loglog(np.mean(psd_grams[ifo], axis=0))
    plt.xlim(20, 512)
    plt.ylim(1e-48)
    plt.show()
    plt.close()

In [ ]:
stride = segment_duration - overlap
csd_segment_offset = int(np.ceil(psd_duration / stride))
after_offset = csd_segment_offset + int(np.ceil(segment_duration / stride))

csd_grams = dict()

for ifo_pair in combinations(gwpy_data.keys(), 2):
    label = "".join(ifo_pair)
    csd_grams[label] = spectral.coarse_grain_spectrogram(
        spectral.density(gwpy_data[ifo_pair[0]], gwpy_data[ifo_pair[1]]),
        delta_f=frequency_resolution,
    )[csd_segment_offset:-after_offset + 1]
    np.log10(abs(csd_grams[label])).plot(ylim=(20, 500))
    plt.show()
    plt.close()

    plt.loglog(abs(np.mean(csd_grams[label], axis=0)))
    plt.xlim(20, 512)
    plt.ylim(1e-50)
    plt.show()
    plt.close()

### Compute coherence

- for each pair of events we wish to compute the coherence $\rho_{IJ} = \frac{C_{IJ}}{\sqrt{S_{I}S_{J}}}$.
- we can marginalize over time or frequency independently to see the time or frequency dependent coherence.

In [ ]:
for ifo_pair in combinations(gwpy_data.keys(), 2):
    label = "".join(ifo_pair)
    coherence = csd_grams[label] / (psd_grams[ifo_pair[0]] * psd_grams[ifo_pair[1]]) ** 0.5
    abs(coherence).plot(ylim=(20, 500))
    plt.show()
    plt.close()
    
    mask = (coherence.frequencies.value >= 20) & (coherence.frequencies.value <= 500)
    values = coherence.value[:, mask]

    plt.plot(coherence.times, np.mean(abs(values), axis=1))
    plt.show()
    plt.close()

    plt.plot(coherence.frequencies[mask], np.mean(abs(values), axis=0))
    plt.show()
    plt.close()

Repeat the test with a single segment used for the psd estimation

In [ ]:
segment_duration = 256
psd_duration = 256
frequency_resolution = 1 / 4
overlap = segment_duration / 2

In [ ]:
gwpy_data = dict()

for ifo in ifos:
    gwpy_data[ifo.name] = TimeSeries(
        ifo.time_domain_strain + common_noise, times=ifo.time_array
    ).fftgram(fftlength=segment_duration, overlap=overlap)
    np.log10(abs(gwpy_data[ifo.name])).plot(ylim=(20, 500))
    plt.show()
    plt.close()

In [ ]:
psd_grams = dict()

for ifo in gwpy_data:
    temp = abs(spectral.coarse_grain_spectrogram(
        spectral.density(gwpy_data[ifo], gwpy_data[ifo]),
        delta_t=psd_duration,
        delta_f=frequency_resolution,
        time_method="running_mean"
    ))
    psd_grams[ifo] = spectral.before_after_average(temp, segment_duration, psd_duration)
    np.log10(psd_grams[ifo]).plot(ylim=(20, 500))
    plt.show()
    plt.close()

    plt.loglog(np.mean(psd_grams[ifo], axis=0))
    plt.xlim(20, 512)
    plt.ylim(1e-48)
    plt.show()
    plt.close()

In [ ]:
stride = segment_duration - overlap
csd_segment_offset = int(np.ceil(psd_duration / stride))
after_offset = csd_segment_offset + int(np.ceil(segment_duration / stride))

csd_grams = dict()

for ifo_pair in combinations(gwpy_data.keys(), 2):
    label = "".join(ifo_pair)
    csd_grams[label] = spectral.coarse_grain_spectrogram(
        spectral.density(gwpy_data[ifo_pair[0]], gwpy_data[ifo_pair[1]]),
        delta_f=frequency_resolution,
    )[csd_segment_offset:-after_offset + 1]
    np.log10(abs(csd_grams[label])).plot(ylim=(20, 500))
    plt.show()
    plt.close()

    plt.loglog(abs(np.mean(csd_grams[label], axis=0)))
    plt.xlim(20, 512)
    plt.ylim(1e-50)
    plt.show()
    plt.close()

In [ ]:
for ifo_pair in combinations(gwpy_data.keys(), 2):
    label = "".join(ifo_pair)
    coherence = csd_grams[label] / (psd_grams[ifo_pair[0]] * psd_grams[ifo_pair[1]]) ** 0.5
    abs(coherence).plot(ylim=(20, 500))
    plt.show()
    plt.close()
    
    mask = (coherence.frequencies.value >= 20) & (coherence.frequencies.value <= 500)
    values = coherence.value[:, mask]

    plt.plot(coherence.times, np.mean(abs(values), axis=1))
    plt.show()
    plt.close()

    plt.plot(coherence.frequencies[mask], np.mean(abs(values), axis=0))
    plt.show()
    plt.close()